In [1]:
from lib import module
import time
import os, itertools
import pickle
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import os
import torch
import argparse
from tqdm import tqdm
import errno
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


base_path = ''
feature_networks_integration = ['clinical', 'cna', 'exp','coe','met','mut'] # datatypes to concatanate node features from
learning_rate = 0.001
hid_size = 128

# optimize for optional feature selection of node features
feature_selection_per_network = [False, False, False,False, False, False]
top_features_per_network = [50, 50, 50,50,50,50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50


# fixed
max_epochs = 500
min_epochs = 200
patience = 30

# fixed to get the same results from the tool each time
random_state = 404

# SUPREME run
print('SUPREME is setting up!')

dataset_name = 'full_data'

path = base_path + "data/" + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cuda:1')

# node_networks = ['exp','coe','met'] # datatypes to use networks from


SUPREME is setting up!


In [2]:
criterion = torch.nn.CrossEntropyLoss()
data_path_node =  base_path + 'data/' + dataset_name +'/'
run_name = 'SUPREME_'+  dataset_name + '_results'
save_path = base_path + run_name + '/'

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

file = base_path + 'data/' + dataset_name +'/labels.pkl'
with open(file, 'rb') as f:
    labels = pickle.load(f)

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
    print('use pre-defined split')
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels)
    print('use random split')
start = time.time()

    
x_lists = []
for netw in feature_networks_integration:
    file = base_path + 'data/' + dataset_name +'/'+ netw +'.pkl'
    with open(file, 'rb') as f:
        feat = pickle.load(f)
        values = feat.values
        x_lists.append(values)
        
new_x = np.concatenate(x_lists,-1)

scaler = StandardScaler()
scaled_new_x = scaler.fit_transform(new_x)
new_x = torch.tensor(scaled_new_x,dtype=torch.float32).to(device)





train_idx,valid_idx = train_test_split(train_valid_idx,test_size=0.2)
train_mask = np.array([i in set(train_idx) for i in range(new_x.shape[0])])
valid_mask = np.array([i in set(valid_idx) for i in range(new_x.shape[0])])
test_mask = np.array([i in set(test_idx) for i in range(new_x.shape[0])])
y_test = pd.DataFrame(labels[test_mask].cpu().numpy()).values.ravel()

in_size = new_x.shape[1]
out_size = torch.unique(labels).shape[0]

use pre-defined split


In [3]:
best_ValidLoss = np.Inf
av_valid_losses = list()    
min_valid_loss = np.Inf
patience_count = 0     


def train():
    model.train()
    optimizer.zero_grad()
    out = model(new_x[train_mask])
    loss = criterion(out, labels[train_mask])
    loss.backward()
    optimizer.step()
    return out


def validate():
    model.eval()
    with torch.no_grad():
        out = model(new_x[valid_mask])
        loss = criterion(out, labels[valid_mask])        
    return loss

def test():
    model.eval()
    with torch.no_grad():
        out = model(new_x[test_mask])
        pred = out.argmax(dim=1)
        y_pred = pred.cpu()
        return accuracy_score(y_pred, y_test)



model = module.MLP(in_size=in_size, hid_size=hid_size, out_size=out_size)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model = model.to(device)
new_x = new_x.to(device)
labels = labels.to(device)
min_valid_loss = np.Inf
patience_count = 0

for epoch in tqdm(range(max_epochs)):
    emb = train()
    this_valid_loss = validate()

    if this_valid_loss < min_valid_loss:
        min_valid_loss = this_valid_loss
        patience_count = 0
        this_emb = emb
    else:
        patience_count += 1

    if epoch >= min_epochs and patience_count >= patience:
        break

av_valid_loss = min_valid_loss.item()

if av_valid_loss < best_ValidLoss:
    best_ValidLoss = av_valid_loss
    test_acc = test()
    print('network ','test acc',test_acc)


 40%|████      | 200/500 [00:01<00:02, 116.10it/s]

network  test acc 0.8297101449275363


In [ ]:
testMLP(node_networks)